In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.


In [2]:
excel_path = "/Users/AlexanderLarsson/documents/VSCode/test_indata.xlsx"
indata_sheet = "test_case_5_whNBD_poisson"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN,60
1,2,Dealer,Deal1,10,10,1,0.95,Poisson,1.0,1.0,15
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,1.0,1.0,6
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,1.0,1.0,5
4,5,Dealer,Deal4,1,10,1,0.95,Poisson,1.0,1.0,4
5,6,Dealer,Deal5,4,10,1,0.95,Poisson,1.0,1.0,7
6,7,Dealer,Deal6,3,10,1,0.95,Poisson,1.0,1.0,6
7,8,Dealer,Deal7,2,10,1,0.95,Poisson,1.0,1.0,5
8,9,Dealer,Deal8,5,10,1,0.95,Poisson,1.0,1.0,9
9,10,Dealer,Deal9,4,10,1,0.95,Poisson,1.0,1.0,7


In [9]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
Q_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Q").to_numpy())
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
demand_mean_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
demand_stdev_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_dealer_variance_arr = demand_stdev_arr**2
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))
h_wh = indataDF.get(indataDF["Type"] == "RDC").get("Holding cost").to_numpy()
R_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Reorder point").to_numpy()
R_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Reorder point").to_numpy())

for id in indataDF.get(indataDF["Type"] == "Dealer").get("Installation id"):
    if indataDF.get(indataDF["Installation id"] == id).get("Demand type").to_numpy() == "Poisson":
        indataDF.loc[indataDF["Installation id"] == id, "Demand stdev"] = math.sqrt(indataDF.loc[outdataDF["Installation id"]== id, "Demand mean"].to_numpy())


Calculating lead time demand mean and std dev for RDC

In [4]:
rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
demand_mean_warehouse = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = demand_mean_warehouse
demand_stdev_warehouse = math.sqrt(sigma2_L * Q_subbatch_size/L_wh)
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = demand_stdev_warehouse
outdataDF.loc[outdataDF["Type"] == "RDC", "Demand variance"] = sigma2_L * Q_subbatch_size/L_wh

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "Dealer","Q, subbatches"] = Q_dealer_arr/Q_subbatch_size



,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Demand variance,"Q, subbatches"
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,1.583581,60,100.0,15.835807,2.507728,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Poisson,1.0,1.000000,15,NaN,NaN,NaN,1.0
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,1.0,1.000000,6,NaN,NaN,NaN,1.0
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,1.0,1.000000,5,NaN,NaN,NaN,1.0
4,5,Dealer,Deal4,1,10,1,0.95,Poisson,1.0,1.000000,4,NaN,NaN,NaN,1.0
5,6,Dealer,Deal5,4,10,1,0.95,Poisson,1.0,1.000000,7,NaN,NaN,NaN,1.0
6,7,Dealer,Deal6,3,10,1,0.95,Poisson,1.0,1.000000,6,NaN,NaN,NaN,1.0
7,8,Dealer,Deal7,2,10,1,0.95,Poisson,1.0,1.000000,5,NaN,NaN,NaN,1.0
8,9,Dealer,Deal8,5,10,1,0.95,Poisson,1.0,1.000000,9,NaN,NaN,NaN,1.0
9,10,Dealer,Deal9,4,10,1,0.95,Poisson,1.0,1.000000,7,NaN,NaN,NaN,1.0


Calculating lead times approximations for dealers

In [5]:
Q_warehouse = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size)
R_warehouse = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Reorder point"))/Q_subbatch_size)

W = waiting_time(negative_inventory(Q_subbatch_size,Q_warehouse,R_warehouse,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)

outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
L_dealer_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = L_dealer_arr
Ldemand_dealer_mean_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = Ldemand_dealer_mean_arr
sqrt_dealer_lead_time_arr = np.sqrt(L_dealer_arr)

Ldemand_dealer_stdev_arr = sqrt_dealer_lead_time_arr*demand_stdev_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] = Ldemand_dealer_stdev_arr


[3.42620443 2.17689613 1.93361753 1.65495522 2.39559528 2.17689613
 1.93361753 2.59593466 2.39559528 2.17689613]
[3.42620443 2.17689613 1.93361753 1.65495522 2.39559528 2.17689613
 1.93361753 2.59593466 2.39559528 2.17689613]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Demand variance,"Q, subbatches",Waiting time,Lead time
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,1.583581,60,100.000000,15.835807,2.507728,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Poisson,1.0,1.000000,15,11.738877,3.426204,NaN,1.0,1.738877,11.738877
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,1.0,1.000000,6,4.738877,2.176896,NaN,1.0,1.738877,4.738877
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,1.0,1.000000,5,3.738877,1.933618,NaN,1.0,1.738877,3.738877
4,5,Dealer,Deal4,1,10,1,0.95,Poisson,1.0,1.000000,4,2.738877,1.654955,NaN,1.0,1.738877,2.738877
5,6,Dealer,Deal5,4,10,1,0.95,Poisson,1.0,1.000000,7,5.738877,2.395595,NaN,1.0,1.738877,5.738877
6,7,Dealer,Deal6,3,10,1,0.95,Poisson,1.0,1.000000,6,4.738877,2.176896,NaN,1.0,1.738877,4.738877
7,8,Dealer,Deal7,2,10,1,0.95,Poisson,1.0,1.000000,5,3.738877,1.933618,NaN,1.0,1.738877,3.738877
8,9,Dealer,Deal8,5,10,1,0.95,Poisson,1.0,1.000000,9,6.738877,2.595935,NaN,1.0,1.738877,6.738877
9,10,Dealer,Deal9,4,10,1,0.95,Poisson,1.0,1.000000,7,5.738877,2.395595,NaN,1.0,1.738877,5.738877


Calculating target fill rate for RDC using induced back order (beta)

In [12]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,demand_mean_arr,demand_stdev_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(demand_mean_arr,demand_mean_warehouse,beta_arr)

target_fill_rate_warehouse = beta_rdc/(beta_rdc + h_wh)

outdataDF.loc[outdataDF["Type"]== "RDC", "Target item fill rate"] = target_fill_rate_warehouse

#Maybe implement to see what the actual fill rate becomes at the warehouse
#if wh_dist == "Normal":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_normal_demand(R_warehouse, Q_warehouse, demand_mean_warehouse, demand_stdev_warehouse)
#elif wh_dist == "Gamma":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = #add here
#elif wh_dist == "NBD":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_compound_poisson_demand(demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse), IL_prob_array_discrete_positive(R_warehouse, Q_warehouse, demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse)))

#outdataDF

0.36898763018965164
[1]


Calculating fill rates for dealers

In [11]:

fill_rate_dealer_arr = []

for i in range(0, len(demand_type_arr)):
    if demand_type_arr[i] == "Normal":
        fill_rate = fill_rate_normal_demand(R_dealer_arr[i], Q_dealer_arr[i], Ldemand_dealer_mean_arr[i], Ldemand_dealer_stdev_arr[i])
        fill_rate_dealer_arr.append(fill_rate)
        print(fill_rate)
    elif demand_type_arr[i] == "Poisson":
        demand_prob_arr = demand_prob_arr_poisson(L_dealer_arr[i], demand_mean_arr[i])
        IL_prob = IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_arr)
        fill_rate_dealer_arr.append(fill_rate_poisson_demand(IL_prob))
    elif demand_type_arr[i] == "NBD":
        print(f"dealer mean arr: {Ldemand_dealer_mean_arr[i]}")
        demand_size_prob_arr = demand_size_arr_logarithmic(demand_mean_arr[i], demand_stdev_arr[i]**2)
        demand_prob_array = demand_prob_arr_negative_binomial(L_dealer_arr[i], demand_mean_arr[i], demand_stdev_arr[i]**2)
        IL_Prob = IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_array)
        fill_rate_NBD = fill_rate_compound_poisson_demand(demand_size_prob_arr, IL_Prob)
        fill_rate_dealer_arr.append(fill_rate_NBD)

outdataDF.loc[outdataDF["Type"]== "Dealer", "Fill rate"] = fill_rate_dealer_arr
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Demand variance,"Q, subbatches",Waiting time,Lead time,Fill rate
0,1,RDC,Johannesburg,10,40,1,0.269533,Normal,10.0,1.583581,60,100.000000,15.835807,2.507728,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.950000,Poisson,1.0,1.000000,15,11.738877,3.426204,NaN,1.0,1.738877,11.738877,0.965493
2,3,Dealer,Deal2,3,10,1,0.950000,Poisson,1.0,1.000000,6,4.738877,2.176896,NaN,1.0,1.738877,4.738877,0.960102
3,4,Dealer,Deal3,2,10,1,0.950000,Poisson,1.0,1.000000,5,3.738877,1.933618,NaN,1.0,1.738877,3.738877,0.967997
4,5,Dealer,Deal4,1,10,1,0.950000,Poisson,1.0,1.000000,4,2.738877,1.654955,NaN,1.0,1.738877,2.738877,0.976514
5,6,Dealer,Deal5,4,10,1,0.950000,Poisson,1.0,1.000000,7,5.738877,2.395595,NaN,1.0,1.738877,5.738877,0.952734
6,7,Dealer,Deal6,3,10,1,0.950000,Poisson,1.0,1.000000,6,4.738877,2.176896,NaN,1.0,1.738877,4.738877,0.960102
7,8,Dealer,Deal7,2,10,1,0.950000,Poisson,1.0,1.000000,5,3.738877,1.933618,NaN,1.0,1.738877,3.738877,0.967997
8,9,Dealer,Deal8,5,10,1,0.950000,Poisson,1.0,1.000000,9,6.738877,2.595935,NaN,1.0,1.738877,6.738877,0.969557
9,10,Dealer,Deal9,4,10,1,0.950000,Poisson,1.0,1.000000,7,5.738877,2.395595,NaN,1.0,1.738877,5.738877,0.952734
